In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install opencv-python

In [ ]:
pip install matplotlib

In [ ]:
import tensorflow as tf
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data' 

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            os.remove(image_path)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')


In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
scaled = data.as_numpy_iterator().next()

In [ ]:
train_size = int(len(data)*.7) -1
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='loss')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_loss')
fig.suptitle('accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

In [ ]:
import cv2

In [ ]:
img = cv2.imread('download.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
# less than 1 its notes pic 
# greater than 1 its normal pic
# 1 means notes 
# 0 means normal

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is notes')
elif yhat>1.0:
    print(f'Predicted class is normal')
else:
    print(f'predicted class is normal')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','deepbinaryimageclassifier.keras'))

In [ ]:
new_model = load_model('models\deepbinaryimageclassifier.keras')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
import cv2 
import shutil
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','deepbinaryimageclassifier.keras'))
new_model = load_model('models\deepbinaryimageclassifier.keras')
new_model.predict(np.expand_dims(resize/255, 0))
img = cv2.imread('images.jpg')

In [ ]:
plt.imshow(img)
plt.show()

In [ ]:
resized = tf.image.resize(img,(256,256))

In [ ]:
plt.imshow(resized.numpy().astype(int))
plt.show()

In [ ]:
yhat = new_model.predict(np.expand_dims(resized/255 ,0))

In [ ]:
yhat

In [ ]:
def move_to_delete_directory(img_file):
    source_path = os.path.join('verifyingdatasetsmixed', img_file)
    destination_path = os.path.join('delete_data', img_file)
    shutil.move(source_path, destination_path)
    print(f"moved {img_file} to delete_data folder")

def move_to_nodelete_directory(img_file):
    source_path = os.path.join('verifyingdatasetsmixed', img_file)
    destination_path = os.path.join('no_delete_data', img_file)
    shutil.move(source_path, destination_path)
    print(f"moved {img_file} to no_delete_data folder")

def move_to_directory(img_file,source_dir,move_dir):
    source_path = os.path.join(source_dir, img_file)
    destination_path = os.path.join(move_dir, img_file)
    shutil.move(source_path, destination_path)
    print(f"moved {img_file} to {move_dir} folder")
    

In [ ]:
image_directory = 'verifyingdatasetsmixed'

image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]
print(image_files)

for img in image_files:
    img_path = os.path.join(image_directory, img)
    img_file = cv2.imread(img_path)
    resized = tf.image.resize(img_file,(256,256))
    yhat = new_model.predict(np.expand_dims(resized/255 ,0))
    if yhat<0.5:
        print(f"the image {img} is not a noteimage")
        move_to_nodelete_directory(img)
    elif yhat>1:
        print(f"the image {img} is expected to be a normal photo")
        move_to_nodelete_directory(img)
    else:
        print(f"the image {img}is a notebook image and moving to delete directory")
        move_to_delete_directory(img)
        

In [ ]:
image_directory_recheck = "delete_data"
image_files_recheck = [f for f in os.listdir(image_directory_recheck) if os.path.isfile(os.path.join(image_directory_recheck, f))]

In [ ]:
for img in image_files_recheck:
    re_img_path = os.path.join(image_directory_recheck,img)
    re_img_file = cv2.imread(re_img_path)
    resized = tf.image.resize(re_img_file,(256,256))
    plt.imshow(re_img_file)
    plt.show()
    re_yhat = new_model.predict(np.expand_dims(resized/255,0))
    if re_yhat<0.5:
        print(f"the image {img} is not a noteimage with yhat {re_yhat}")
        move_to_directory(img,'delete_data','no_delete_data')
    elif yhat>1:
        print(f"the image {img} is expected to be a normal photo with yhat {re_yhat}")
        move_to_directory(img,'delete_data','no_delete_data')
    else:
        print(f"the image {img}is a notebook image and moving to delete directory with yhat {re_yhat}")
        move_to_directory(img,'delete_data','delete_data')
        